# Assignment 8




In [1]:
import pandas as pd
import numpy as np
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec  # Import Word2Vec from Gensim
from sklearn.svm import SVC  


In [2]:
data=pd.read_csv("D:/ML/spam.csv",encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
def pre_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
        stemmer = SnowballStemmer("english")
        words += (stemmer.stem(i)) + " "
    return words

In [5]:
textFeatures = data['text'].copy()
textFeatures = textFeatures.apply(pre_process)

In [6]:
tokenized_text = [text.split() for text in textFeatures]


In [7]:
# Train a Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, sg=0)


In [8]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(data['class'])

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(tokenized_text, Y, test_size=0.3, random_state=111)


In [10]:
# Create a function to convert text to Word2Vec embeddings
def text_to_word2vec(text, model):
    # Get Word2Vec embeddings for each word in the text and average them
    word_vectors = [model.wv[word] for word in text if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Return zeros for out-of-vocabulary words


In [11]:
# Convert text data to Word2Vec embeddings
X_train = [text_to_word2vec(text, word2vec_model) for text in X_train]
X_test = [text_to_word2vec(text, word2vec_model) for text in X_test]

In [12]:
# Convert lists to NumPy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

In [13]:
# Create and train a classifier (SVM in this example)
classifier = SVC(kernel='linear')
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [14]:
# Evaluate the classifier on the test set
accuracy = classifier.score(X_test, Y_test)
print("Test accuracy:", accuracy)

Test accuracy: 0.861244019138756
